In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import pandas as pd
import os
import spfuncs as spf
from astropy.io import fits
from astropy.table import Table
from scipy.io import readsav
from scipy.stats import chi2
from scipy.interpolate import UnivariateSpline
from IPython.display import clear_output
from astropy.convolution import Gaussian1DKernel, convolve_fft
from scipy.stats import binned_statistic

In [236]:
##### Global variables #####
# global variable for instrument bits
INST_BITS = {'G140L':1,
             'G140M':2,
             'G230L':4,
             'G430L':8,
             'X-RAY':16,
             'proxy':32,
             'lya':64,
             'EUV':128,
             'BT-Settl':256,
             'APEC':512}

In [237]:
##### FUNCTIONS #####
# Search for a spectrum in the given grating directory. If found, updates given grating dict and returns true
# Requires spectrum be named x1d_coadd.fits
def look_for_data(star,grating_to_find,grating_dict):
    # Search through the directories in the Star directory and try to find the given grating
    
    print('Searching for',grating_to_find)
    try:
        for root,dirs,files in os.walk('./%s'%(star)):
            if(root == './%s\\%s'%(star,grating_to_find)):
                print('Found directory: %s'%(root))
                # Try to find the coadded spectrum in the grating directory
                try:
                    with fits.open('%s/x1d_coadd.fits'%(root)) as hdul:
                        wave = hdul[1].data['WAVELENGTH']
                        flux = hdul[1].data['FLUX']
                        err = hdul[1].data['ERROR']
                        inst = INST_BITS[grating_to_find]*np.ones(len(wave))
                    print('Retrieved',grating_to_find)
                    grating_dict[grating_to_find] = np.array([wave,flux,err,inst])
                    return True
                except:
                    print('Error finding spectrum')
    except:
        print('Error finding star directory')
    
    print('Did not find directory')
    return False

def minimize_scale_factor(ref,spectrum,init_factor,search_range):
    # Minimizes the scale factor [(F_obs - F_proxy)/E_obs]^2. Returns the best scale factor 
    #      and the minimum "chi^2" value
    # INPUT:
    # ref = reference spectrum. numpy array including wavelength, flux, error [wave,flux,err]
    # spectrum = spectrum to be scaled. 1d numpy array containing ONLY flux.
    # init_factor = initial scaling factor
    # search_range = order of magnitude to search around init_factor. ie: search_range = 2 will search for 
    #                factors between init_factor*10^(-2)-init_factor*10^(+2)
    # OUTPUT:
    # alpha scaling value and minimum "chi squared" factor
    
    scale_factors = np.linspace(init_factor*10**(-search_range),init_factor*10**(search_range),num=5000)
    #print('Scale factor lims',min(scale_factors),max(scale_factors)) # debugging
    best_factor = 1
    min_chi2 = np.inf
    
    for alpha in scale_factors:
        scaled_flux = spectrum*alpha
        chi2 = np.sum(((ref[1]-scaled_flux)/ref[2])**2)/(len(ref[0])-1)
        
        if(chi2 < min_chi2):
            min_chi2 = chi2
            best_factor = alpha
            
    return best_factor,min_chi2

def scale_spectrum(ref,spectrum,init_factor=1,coarse_range=2,fine_range=1,end='high'):
    # Takes a reference spectrum and a spectrum to be scaled. Calls minimize_scale_factor and returns the best 
    #        scale factor to match the spectrum to the reference.
    # INPUT:
    # ref, spectrum, init_factor = same as minimize_scale_factor()
    # coarse_range = value of search_range to be passed to minimize_scale_factor() for the initial search.
    #                This is intended to search a braod range to avoid local minima
    # fine_range = value of search_range to be passed to minimize_scale_factor() for the 2nd search.
    #               This is intended to find a more precise minimum around the result from the coarse search
    # end = which end of the reference spectrum are you scaling to. Must be 'low' or 'high' for blue 
    #               and red end respectively
    # OUTPUT:
    # scaled spectrum in form [wavelength, flux*alpha, error*alpha]
    # alpha scaling factor
    # minimum "chi squared" factor
    
    # Get the region where the spectrum and reference overlap
    if(end.lower()=='high'):
        try:
            ref_reg = ref[0] > spectrum[0][0]
            spec_reg = spectrum[0] < ref[0][-1]
            
            ref = ref[:,ref_reg]
            spectrum = spectrum[:,spec_reg]
        except:
            raise IndexError('For end=\'high\' the spectrum must overlap with the red end of the reference')
    elif(end.lower()=='low'):
        try:
            ref_reg = ref[0] < spectrum[0][-1]
            spec_reg = spectrum[0] > ref[0][0]
            
            ref = ref[:,ref_reg]
            spectrum = spectrum[:,spec_reg]
        except:
            raise IndexError('For end=\'low\' the spectrum must overlap with the blue end of the reference')
    else:
        raise Exception('Keyword \'end\' must be \'high\' or \'low\'')
        
    print('Fitting region:\t%0.1f-%0.1f'%(min(spectrum[0]),max(spectrum[0])))
        
    # Interpolate the spectrum with greater A/pix onto the wavelength grid of the other
    ref_ang_per_pix = np.nanmean(np.diff(ref[0])/len(ref[0]))
    spectrum_ang_per_pix = np.nanmean(np.diff(spectrum[0])/len(ref[0]))
    
    if((ref_ang_per_pix < spectrum_ang_per_pix) or (ref_ang_per_pix == spectrum_ang_per_pix)):
        spectrum_interp = np.interp(x=ref[0],xp=spectrum[0],fp=spectrum[1])
        
        # Run the minimization routine with a broad, coarse search
        scale_factor_coarse,_ = minimize_scale_factor(ref,spectrum_interp,init_factor,search_range=coarse_range)
        # Run the minimization routine with a fine search around the initial factor
        scale_factor,chi2 = minimize_scale_factor(ref,spectrum_interp,scale_factor_coarse,search_range=fine_range)
        
    elif(spectrum_ang_per_pix < ref_ang_per_pix):
        ref_interp = np.interp(x=spectrum[0],xp=ref[0],fp=ref[1])
        ref_err_interp = np.interp(x=spectrum[0],xp=ref[0],fp=ref[2])
        
        ref_to_pass = np.array([spectrum[0],ref_interp,ref_err_interp])
        
        # Run the minimization routine with a broad, coarse search
        scale_factor_coarse,_ = minimize_scale_factor(ref_to_pass,spectrum[1],init_factor,search_range=coarse_range)
        # Run the minimization routine with a fine search around the initial factor
        scale_factor,chi2 = minimize_scale_factor(ref_to_pass,spectrum[1],scale_factor_coarse,search_range=fine_range)
        
    print('--- RESULTS ---')
    print('Best scale factor:\t%0.4e'%(scale_factor))
    print('Minimum chi square:\t%0.4e'%(chi2))
    
    return scale_factor,chi2

# Bin the instrument bits. After binning the spectrum, if there are data in a single bin coming from multiple sources
#     get_inst_bins() will add the instrument bits to indicate the separate sources
def get_inst_bins(inst_array):
    # INPUT: 1D instrument bit array from the spectrum being binned
    # OUTPUT: scalar value representing the sum of the instrument bits in the bin
    bin_val = np.sum(np.unique(inst_array))
    return bin_val

# Star & Proxy Parameters

In [5]:
STAR = 'L-98-59'.upper()
PROXY_NAME = 'GJ 832'
STELLAR_TYPE='M'
STELLAR_RADIUS_SOL = 0.303
STELLAR_DIST_PC = 10
pc2au = 206265

# Read FITS files from star directory

In [6]:
grating_exists = {'G140L':False,'G140M':False,'G230L':False,'G430L':False}
gratings = {}

for grating_name in ['G140L','G140M','G230L','G430L']:
    grating_exists[grating_name] = look_for_data(STAR,grating_name,gratings)
    
# Retrieve proxy star spectrum
try:
    if(PROXY_NAME=='Sun'):
        solar_dat = readsav('./Proxy stars/Solar/solar-data.idlsav')
        solar_wave = solar_dat['wave'][:,0]*10
        solar_flux = solar_dat['flux'][:,0]*100
        inst = INST_BITS['proxy']*np.ones(len(solar_wave))
        
        proxy = np.array([solar_wave,solar_flux,np.zeros(len(solar_wave)),inst])
        print('Retrieved Solar proxy')
    else:
        for root,dirs,files in os.walk('./Proxy stars'):
            if(root == './Proxy stars\\%s'%(PROXY_NAME)):
                print('Found directory: %s'%(root))
                
                with fits.open('./Proxy stars/%s/%s'%(PROXY_NAME,files[0])) as hdul:
                    wave = hdul[1].data['WAVELENGTH']
                    flux = hdul[1].data['FLUX']
                    err = hdul[1].data['ERROR']
                    inst = INST_BITS['proxy']*np.ones(len(wave))
                    
                    proxy = np.array([wave,flux,err,inst])
                print('Retrieved proxy')
except:
    print('Could not retrieve proxy')
    
print('\nGratings list:')
print(grating_exists)

Searching for G140L
Found directory: ./L-98-59\G140L
Retrieved G140L
Searching for G140M
Found directory: ./L-98-59\G140M
Retrieved G140M
Searching for G230L
Found directory: ./L-98-59\G230L
Retrieved G230L
Searching for G430L
Found directory: ./L-98-59\G430L
Retrieved G430L
Found directory: ./Proxy stars\GJ 832
Retrieved proxy

Gratings list:
{'G140L': True, 'G140M': True, 'G230L': True, 'G430L': True}


#### Special cases: ie: COS, uncommmon STIS gratings, etc, do manually

# Examine data and select good regions

In [8]:
for grating_name in gratings:
    print('Working with',grating_name)
    grating = gratings[grating_name]
    
    # Get good error region for plottign SNR
    good_err_reg = (grating[2] != 0)
    
    fig,ax = plt.subplots(2,1,figsize=(10,8),tight_layout=True,sharex=True,num=grating_name)
    fig.suptitle(grating_name)
    ax[0].plot(grating[0],grating[1])
    ax[1].plot(grating[0][good_err_reg],grating[1][good_err_reg]/grating[2][good_err_reg])
    ax[1].hlines(y=3,xmin=min(grating[0]),xmax=max(grating[0]),color='k',linestyle='--')
    ax[1].set_ylim([-3,20])
    
    ax[1].set_xlabel('Wavelength [A]')
    ax[0].set_ylabel('Flux [erg / s / cm^2 / A]')
    ax[1].set_ylabel('S/N')
    
    clicks = np.array(fig.ginput(n=-1,timeout=0))[-2::][:,0]
    
    selected_region = (grating[0] > clicks[0]) & (grating[0] < clicks[1])
    print('Keeping wavelengths from:  %0.2f-%0.2f'%(grating[0][selected_region][0],grating[0][selected_region][-1]))
    
    gratings[grating_name] = grating[:,selected_region]

Working with G140L
Keeping wavelengths from:  1162.50-1699.00
Working with G140M
Keeping wavelengths from:  1197.60-1247.10
Working with G230L
Keeping wavelengths from:  2588.50-3140.50
Working with G430L
Keeping wavelengths from:  3678.00-5694.50


## Retrieve Ly$\alpha$ profile & BT-Settl

In [9]:
# Lya

try:
    lya_data = pd.read_csv('./%s/lya_reconstruction/%s_G140M_MCMC_results.csv'%(STAR,STAR))
    lya = np.array([lya_data['wave_lya'],lya_data['lya_intrinsic_median']])
    print('Retrieved MCMC reconstruction')
except:
    try:
        lya_data = pd.read_csv('./%s/lya_reconstruction/%s_lya_intrinsic_profile.csv'%(STAR,STAR))
        lya = np.array([lya_data['wave'],lya_data['flux']])
        print('Retrieved Mg II reconstruction')
    except:
        print('No Lyman-alpha profile found')
        
# BT-Settl
try:
    settl_dat = Table.read('./%s/interp_spectra/%s_phoenix_interpolated.ecsv'%(STAR,STAR))
    settl = np.array([settl_dat['WAVELENGTH'],
                      settl_dat['FLUX']*settl_dat.meta['NORMFAC'],
                      np.zeros(len(settl_dat['FLUX'])),
                      np.ones(len(settl_dat['WAVELENGTH']))*INST_BITS['BT-Settl']])
    print('Retrieved BT-Settl')
except:
    print('No SETTL data found')

Retrieved MCMC reconstruction
Retrieved BT-Settl


# Convolve & Scale BT-Settl

#### Convolve BT-Settl to G430L Resolution

In [10]:
# Convolve BT-Settl to G430L resolution (2.73A/pix [STIS handbook])
settl_conv_flux = spf.convolve_gaussian(settl[0],settl[1],2.73)
settl_conv = np.array([settl_conv_flux[0],settl_conv_flux[1],np.zeros(len(settl_conv_flux[0]))])

#### Scale convolved BT-Settl to G430L data

In [231]:
# Select the region of the settl spectrum to scale and run the minimization routine. Recommend >5000A
settl_to_fit = settl_conv[0] > 5000

settl_alpha,settl_chi2 = scale_spectrum(ref=gratings['G430L'],spectrum=settl_conv[:,settl_to_fit],
                                                init_factor=1,coarse_range=2,fine_range=0.5,end='high')

# Remember to scale the error, too!
scaled_settl = np.array([settl_conv[0],
                         settl_conv[1]*settl_alpha,
                         settl_conv[2]*settl_alpha,
                         np.ones(len(settl_conv[0]))*INST_BITS['BT-Settl']])

fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True);
[x.set_linewidth(1.5) for x in ax.spines.values()];
ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=12);
ax.tick_params(which='minor',length=4,width=1.0);
ax.minorticks_on();
ax.plot(gratings['G430L'][0],gratings['G430L'][1],label='G430L');
ax.plot(settl_conv[0],settl_conv[1]*settl_alpha,label=r'Scaled SETTL: $\alpha$=%0.4f'%(settl_alpha));
ax.plot(settl_conv[0],settl_conv[1],label='Unscaled SETTL');
ax.legend(fontsize=12);

ax.set_xlim([min(gratings['G430L'][0]),max(gratings['G430L'][0])]);
ax.set_xlabel(r'Wavelength [$\AA$]',fontsize=14);
ax.set_ylabel(r'Flux density [erg s$^{-1}$ cm${-2}$ $\AA^{-1}$]',fontsize=14);

Fitting region:	5004.3-5692.5
--- RESULTS ---
Best scale factor:	9.1889e-01
Minimum chi square:	7.1384e+02


#### Scale proxy spectrum to G230L

In [12]:
# Select the region of the proxy spectrum to scale and run the minimization routine, assuming we have a G230L spectrum
# IGNORES MGII LINE
if(grating_exists['G230L']):
    proxy_to_fit = (proxy[0] > gratings['G230L'][0][0]) & (proxy[0] < gratings['G230L'][0][-1])
    mg2_reg = [2770,2820]
    
    proxy_mg = (proxy[0] > mg2_reg[0]) & (proxy[0] < mg2_reg[1])
    stis_mg = (gratings['G230L'][0] > mg2_reg[0]) & (gratings['G230L'][0] < mg2_reg[1])
    

    if(PROXY_NAME=='Sun'):
        
        # If the proxy is the Sun, scale it by 1e-14 first (approximate order of magnitude of most of our fluxes)
        #     to get the minimization to converge
        init_proxy = proxy[:,proxy_to_fit^proxy_mg]
        init_proxy[1] = init_proxy[1]*1e-14

        proxy_alpha,proxy_chi2 = scale_spectrum(ref=gratings['G230L'][:,~stis_mg],spectrum=init_proxy,
                                                        init_factor=1,coarse_range=3,fine_range=0.5,end='low')

        proxy_alpha = proxy_alpha*1e-14
        
        scaled_proxy = np.array([proxy[0],
                                 proxy[1]*proxy_alpha,
                                 proxy[2]*proxy_alpha,
                                 np.ones(len(proxy[0]))*INST_BITS['proxy']])
        
        fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True)
        ax.plot(gratings['G230L'][0],gratings['G230L'][1],label='G230L')
        ax.plot(scaled_proxy[0],scaled_proxy[1],label=r'Scaled proxy: $\alpha$=%0.4e'%(proxy_alpha))
        ax.legend()

        ax.set_xlim([min(proxy[0][proxy_to_fit]),max(gratings['G230L'][0])])

        ax.set_xlabel(r'Wavelength [$\AA$]')
        ax.set_ylabel(r'Flux density [erg s$^{-1}$ cm${-2}$ $\AA^{-1}$]')

        print('EXCEPTION: SOLAR PROXY')
        print('Acutal scale factor:\t%0.4e'%(proxy_alpha))
        
    else:
        proxy_alpha,proxy_chi2 = scale_spectrum(ref=gratings['G230L'][:,~stis_mg],
                                                             spectrum=proxy[:,proxy_to_fit^proxy_mg],
                                                        init_factor=0.01,coarse_range=3,fine_range=0.5,end='low')

        scaled_proxy = np.array([proxy[0],
                                 proxy[1]*proxy_alpha,
                                 proxy[2]*proxy_alpha,
                                 np.ones(len(proxy[0]))*INST_BITS['proxy']])
        
        
        fig,ax = plt.subplots(1,1,figsize=(10,6),tight_layout=True);
        ax.plot(gratings['G230L'][0],gratings['G230L'][1],label='G230L');
        ax.plot(scaled_proxy[0],scaled_proxy[1],label=r'Scaled proxy: $\alpha$=%0.4e'%(proxy_alpha));
        ax.legend();

        ax.set_xlim([min(proxy[0][proxy_to_fit]),max(gratings['G230L'][0])]);

        ax.set_xlabel(r'Wavelength [$\AA$]');
        ax.set_ylabel(r'Flux density [erg s$^{-1}$ cm${-2}$ $\AA^{-1}$]');

Fitting region:	2589.5-3140.3
--- RESULTS ---
Best scale factor:	3.7952e-02
Minimum chi square:	1.9975e+00


# Get EUV bins

In [94]:
#### EUV bin relations from Linsky et al (2014). Use integrated lyman-a flux to calculate 100-1170A flux density

print('Stellar type:',STELLAR_TYPE)
print('Distance: %dpc'%(STELLAR_DIST_PC))

### Get Lya flux & scale to 1AU ###
lya_scale_fac = (STELLAR_DIST_PC*pc2au)**2
int_lya_flux = np.trapz(x=lya[0],y=lya[1])
# Proxy lya flux
#int_lya_flux = np.trapz(x=proxy[0][(proxy[0] > 1214) & (proxy[0] < 1217.4)],y=proxy[1][(proxy[0] > 1214) & (proxy[0] < 1217.4)]*proxy_alpha)
print('Integrated lya flux:\t\t%0.4e erg/s/cm^2'%(int_lya_flux))

int_lya_flux = int_lya_flux*lya_scale_fac
print('Integrated lya flux at 1AU:\t%0.4e erg/s/cm^2'%(int_lya_flux))

# Use Linsky 2014 relations for EUV bins. Relations hold for F-M stars
SUPPORTED_STELLAR_TYPES = ['M','K','G','F']

# Bins 100-400 are different for M stars and F-K stars
if(STELLAR_TYPE.upper() in SUPPORTED_STELLAR_TYPES):
    if(STELLAR_TYPE.upper()=='M'):
        bin_100_200 = 10**(-0.491)*int_lya_flux/100/lya_scale_fac
        bin_200_300 = 10**(-0.548)*int_lya_flux/100/lya_scale_fac
        bin_300_400 = 10**(-0.602)*int_lya_flux/100/lya_scale_fac

    elif(STELLAR_TYPE.upper()=='K' or STELLAR_TYPE.upper()=='F' or STELLAR_TYPE.upper()=='G'):
        bin_100_200 = 10**(-1.357)*int_lya_flux**(0.344+1)/100/lya_scale_fac
        bin_200_300 = 10**(-1.300)*int_lya_flux**(0.309+1)/100/lya_scale_fac
        bin_300_400 = 10**(-0.882)*int_lya_flux/100/lya_scale_fac
    else:
        print('How did you get here?')

    # Bins 400-1170 are the same for all stars.
    bin_400_500 = 10**(-2.294)*int_lya_flux**(0.258+1)/100/lya_scale_fac
    bin_500_600 = 10**(-2.098)*int_lya_flux**(0.572+1)/100/lya_scale_fac
    bin_600_700 = 10**(-1.920)*int_lya_flux**(0.240+1)/100/lya_scale_fac
    bin_700_800 = 10**(-1.894)*int_lya_flux**(0.518+1)/100/lya_scale_fac
    bin_800_912 = 10**(-1.811)*int_lya_flux**(0.764+1)/112/lya_scale_fac
    bin_912_1170 = 10**(-1.004)*int_lya_flux**(0.065+1)/258/lya_scale_fac

    bin_edges = np.array([100,200,300,400,500,600,700,800,912,1170])
    bin_fluxes = np.array([bin_100_200,bin_200_300,bin_300_400,bin_400_500,
                          bin_500_600,bin_600_700,bin_700_800,bin_800_912,bin_912_1170])

    # Define a new wavelength array for the EUV bins to replace the proxy data
    euv_reg = (proxy[0] > 100) & (proxy[0] < 1170)
    EUV_wave_array = proxy[0][euv_reg]
    EUV_flux_array = np.ones(len(EUV_wave_array))
    EUV_inst_array = np.ones(len(EUV_wave_array))*INST_BITS['EUV']

    for i in range(len(bin_fluxes)):
        print('Bin %d-%d:\t%0.4e erg/s/cm^2/A'%(bin_edges[i],bin_edges[i+1],bin_fluxes[i]))
        current_bin = (EUV_wave_array >= bin_edges[i]) & (EUV_wave_array < bin_edges[i+1])
        EUV_flux_array[current_bin] = bin_fluxes[i]

    fig = plt.figure(figsize=(12,8),tight_layout=True);
    ax = fig.add_subplot();
    [x.set_linewidth(1.5) for x in ax.spines.values()];
    ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=12);
    ax.tick_params(which='minor',length=4,width=1.5);
    ax.minorticks_on()
    plt.step(EUV_wave_array,EUV_flux_array);
    plt.title('EUV flux bins',fontsize=15);
    plt.yscale('log');
    plt.grid(which='both',axis='both',alpha=0.4);
    plt.xlabel('Wavelength [A]',fontsize=14);
    plt.ylabel('Flux density [erg / s / cm^2 / A]',fontsize=14);
    
else:
    print('Unsupported stellar type. Skipping EUV bins')
    
    
EUV_data = np.array([EUV_wave_array,
                     EUV_flux_array,
                     np.zeros(len(EUV_wave_array)),
                     EUV_inst_array])

Stellar type: M
Distance: 10pc
Integrated lya flux:		5.4100e-14 erg/s/cm^2
Integrated lya flux at 1AU:	2.3017e-01 erg/s/cm^2
Bin 100-200:	1.7466e-16 erg/s/cm^2/A
Bin 200-300:	1.5318e-16 erg/s/cm^2/A
Bin 300-400:	1.3527e-16 erg/s/cm^2/A
Bin 400-500:	1.8819e-18 erg/s/cm^2/A
Bin 500-600:	1.8633e-18 erg/s/cm^2/A
Bin 600-700:	4.5718e-18 erg/s/cm^2/A
Bin 700-800:	3.2266e-18 erg/s/cm^2/A
Bin 800-912:	2.4299e-18 erg/s/cm^2/A
Bin 912-1170:	1.8885e-17 erg/s/cm^2/A


# Get X-ray spectrum

In [223]:
########### CONSTANTS ##############
c = 3e18 # A/s
h = 4.1357e-18 # keV s
erg = 1/6.2415e8 # 1 erg = 6.2e8 keV
####################################

# Get the effective area. Only use this if you were dumb and forgot to use setplot area in XSPEC
get_aeff = False

if(get_aeff):
    d = 3.09e18*10 # distance in cm

    with fits.open('./L-98-59/XMM/flare_analysis/F301/src_quiesfilt_0.3-10kev_arf.fits') as hdul:
        resp_energy = hdul[1].data['ENERG_LO']
        a_eff = hdul[1].data['SPECRESP']


    closest_energy = np.zeros(len(x_ray_obs['energy']),dtype=int)


    #### Get the closest energy value to the observed bins & divide by the effective area at that energy
    for i in range(len(x_ray_obs['energy'])):
        closest_energy[i] = int(np.argmin(np.abs(resp_energy-x_ray_obs['energy'][i])))
        print('Energy:\t%0.2fkeV\nA_eff:\t%0.1fcm^2'%(resp_energy[closest_energy[i]],a_eff[closest_energy[i]]))

    fig,ax = plt.subplots(1,1,tight_layout=True,figsize=(9,7));
    [x.set_linewidth(1.2) for x in ax.spines.values()]
    ax.tick_params(which='both',direction='in',top=True,right=True,length=7,width=1.5,labelsize=12);
    ax.tick_params(which='minor',length=4,width=1.0);
    ax.minorticks_on();
    ax.plot(APEC_model['energy'],APEC_model['model_flux'],label='APEC plasma model',zorder=-1);
    ax.errorbar(x=x_ray_obs['energy'],y=x_ray_obs['counts'],
                xerr=x_ray_obs['bin_width'],yerr=x_ray_obs['counts_err'],
                linestyle='',label='X-ray observation',zorder=0);
    ax.set_ylabel(r'Counts / s / cm$^2$ / keV',fontsize=14);
    ax.set_xlabel('Energy [keV]',fontsize=14);
    ax.legend(fontsize=12)

#### TODO
#### COME UP WITH A WAY TO AUTOMATE RETRIEVAL OF X-RAY DATA & XSPEC MODELS

####### x-ray model CSV contains ########
# Energy [keV]  |  energy bin width [keV]  |  total counts  |  counts error  |  photon flux [photon/s/cm^2/keV]
#########################################


x_ray_data = pd.read_csv('./L-98-59/XMM/flare_analysis/F301/x_ray_model.csv',delimiter=',')

end_of_data = 0
start_of_APEC = 0
for row,index in zip(x_ray_data['energy'],range(len(x_ray_data['energy']))):
    print(index,row)
    if(row.lower()=='break'):
        # Subtract 1 from index because pandas range is inclusive
        end_of_data = index-1
        start_of_APEC = index+1
        print('Data ends at:',index)
        break
        
x_ray_obs = x_ray_data.loc[0:end_of_data].astype(float)
print(x_ray_obs)
APEC_model = x_ray_data.loc[start_of_APEC::].astype(float)
print(APEC_model)


obs_energy = np.array(x_ray_obs['energy'])[::-1]

obs_wave = np.array(h*c/obs_energy)
obs_flux = np.array(x_ray_obs['counts'][::-1]*obs_energy)*erg*obs_energy**2/(h*c)
obs_err = np.array(x_ray_obs['counts_err'][::-1]*obs_energy*erg*obs_energy**2/(h*c))


apec_energy = np.array(APEC_model['energy'])[::-1]
apec_wave = h*c/apec_energy
apec_flux = np.array(APEC_model['model_flux'][::-1])*erg*apec_energy**2/(h*c)

fig,ax = plt.subplots(1,1,figsize=(10,7),tight_layout=True)
ax.errorbar(x=obs_wave,
            y=obs_flux,
            xerr=np.sqrt((-h*c/obs_energy**2)**2*x_ray_obs['bin_width'][::-1]**2),
            yerr=obs_err,linestyle='',label=STAR,zorder=2)
ax.plot(apec_wave,apec_flux,label='APEC',zorder=1)
ax.plot(wave,flux*proxy_alpha,label='Scaled proxy',zorder=0)
ax.set_xlabel('Wavelength [Å]')
ax.set_ylabel('erg / s / cm^2 / A')

ax.set_yscale('log')
ax.minorticks_on()
ax.legend()

fig.suptitle(STAR)

apec_region1 = (apec_wave > 5.) & (apec_wave < obs_wave[1])
apec_region2 = (apec_wave > obs_wave[-1]) & (apec_wave < 100)
apec_region = apec_region1+apec_region2

apec_data = np.array([apec_wave[apec_region],
                      apec_flux[apec_region],
                      np.zeros(np.sum(apec_region)),
                      np.ones(np.sum(apec_region))*INST_BITS['APEC']])


obs_data = np.array([obs_wave[1::],
                     obs_flux[1::],
                     obs_err[1::],
                     np.ones(len(obs_wave[1::]))*INST_BITS['X-RAY']])

xray_data = np.concatenate((apec_data,obs_data),axis=1)

Energy:	0.36keV
A_eff:	448.7cm^2
Energy:	0.41keV
A_eff:	538.3cm^2
Energy:	0.51keV
A_eff:	757.8cm^2
Energy:	0.62keV
A_eff:	797.1cm^2
Energy:	0.73keV
A_eff:	915.1cm^2
Energy:	0.82keV
A_eff:	979.0cm^2
Energy:	0.94keV
A_eff:	1037.5cm^2
Energy:	1.04keV
A_eff:	1073.4cm^2
Energy:	1.72keV
A_eff:	1098.1cm^2
Energy:	3.73keV
A_eff:	765.1cm^2
0 0.361546516
1 0.413447976
2 0.511106014
3 0.620116472
4 0.727509499
5 0.818952501
6 0.938667536
7 1.03608
8 1.71543002
9 3.74010992
10 BREAK
Data ends at: 10
     energy  bin_width        counts    counts_err    model_flux
0  0.361547   0.010441  1.431852e-04  7.793862e-05  1.831831e-05
1  0.413448   0.041460  2.941793e-05  1.770078e-05  1.692916e-05
2  0.511106   0.056198  1.684862e-05  1.132283e-05  1.653663e-05
3  0.620116   0.052812  1.689382e-05  1.016502e-05  2.343895e-05
4  0.727509   0.054581  2.115721e-05  9.090589e-06  2.108925e-05
5  0.818953   0.036862  2.318555e-05  1.126630e-05  1.456621e-05
6  0.938668   0.082853  8.176828e-06  4.674087e-06  

# Put the pieces together

In [234]:
#### Obs UV data
# gratings['G230L'],  gratings['G430L']
STIS_data = np.concatenate((gratings['G230L'],gratings['G430L']),axis=1)
plt.plot(STIS_data[0],STIS_data[1],label='STIS')

#### Proxy data
# scaled_proxy
proxy_region1 = (scaled_proxy[0] > 1170.) & (scaled_proxy[0] < gratings['G230L'][0][0])
proxy_region2 = (scaled_proxy[0] > gratings['G230L'][0][-1]) & (scaled_proxy[0] < gratings['G430L'][0][0])
proxy_region = proxy_region1+proxy_region2

proxy_data = scaled_proxy[:,proxy_region]

plt.plot(proxy_data[0],proxy_data[1])

#### BT-Settl
# scaled_settl
settl_region = (scaled_settl[0] > gratings['G430L'][0][-1]) & (scaled_settl[0] < 55000)

settl_data = scaled_settl[:,settl_region]

plt.plot(settl_data[0],settl_data[1])

#### EUV data
# EUV_data
plt.plot(EUV_data[0],EUV_data[1])

#### X-ray data
# xray_data
plt.errorbar(x=xray_data[0],y=xray_data[1],yerr=xray_data[2])

# Combine each data set into a single array
combined_spectrum_nolines = np.concatenate((xray_data,
                                           EUV_data,
                                           proxy_data,
                                           STIS_data,
                                           settl_data),axis=1)

# Get the sorted indices of the wavelength array using argsort and use that to sort the entire combined array
combined_spectrum_nolines = combined_spectrum_nolines[:,np.argsort(combined_spectrum_nolines[0])]

In [235]:
fig,ax = plt.subplots(3,1,figsize=(9,9),tight_layout=True,sharex=True)
for i in ax:
    [x.set_linewidth(1.5) for x in i.spines.values()]
    i.tick_params(which='both',top=True,right=True,length=7,width=1.5,labelsize=14)
    i.tick_params(which='both',length=4,width=1.)

pos_flux = combined_spectrum_nolines[2] > 0

ax[0].plot(combined_spectrum_nolines[0],combined_spectrum_nolines[1])
ax[1].plot(combined_spectrum_nolines[0][pos_flux],combined_spectrum_nolines[1][pos_flux]/combined_spectrum_nolines[2][pos_flux])
ax[2].plot(combined_spectrum_nolines[0],combined_spectrum_nolines[3])

# Test Instrument binning

In [239]:
test_wave = gratings['G140L'][0]
test_flux = gratings['G140L'][1]
test_inst = gratings['G140L'][3]

test_inst[100:102] = INST_BITS['proxy']

ax1 = plt.subplot(211)
plt.plot(test_wave,test_flux)
plt.subplot(212,sharex=ax1)
plt.plot(test_wave,test_inst)

In [240]:
num_bins = test_wave[-1]-test_wave[0]

test_flux_binned,flux_bins,_ = binned_statistic(x=test_wave,values=test_flux,bins=num_bins,statistic='mean')
test_inst_binned,inst_bins,_ = binned_statistic(x=test_wave,values=test_inst,bins=num_bins,statistic=get_inst_bins)

In [241]:
ax1 = plt.subplot(211)
plt.plot(flux_bins[0:-1],test_flux_binned)
plt.subplot(212,sharex=ax1)
plt.plot(inst_bins[0:-1],test_inst_binned)
plt.plot(test_wave,test_inst)

# GJ 832 Spectrum

In [96]:
with fits.open('./Proxy stars/GJ 832/hlsp_muscles_multi_multi_gj832_broadband_v22_var-res-sed.fits') as hdul:
    plt.figure()
    plt.semilogy(hdul[1].data['WAVELENGTH'],hdul[1].data['FLUX'])
    
    gjwave = hdul[1].data['WAVELENGTH']
    gjflux = hdul[1].data['FLUX']
    
gj_832_apec = pd.read_csv('./GJ_832_apec.csv')
#
#plt.plot(h*c/gj_832_apec['energy'],gj_832_apec['model_flux']*erg*gj_832_apec['energy']**2/(h*c)*(9.35e-15/9.365e-10))
#
plt.figure()
plt.plot(np.diff(gjwave[(gjwave > 20) & (gjwave < 100)]))